In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

# Load the movie Lens class
ml = MovieLens()

# Algorithm

In [2]:
from surprise import Dataset, Reader, accuracy, AlgoBase, PredictionImpossible, NormalPredictor
from surprise.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# Load the ratings dataset
ratings = ml.ratings.copy()
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
print(ratings['rating'].nunique())
list(ratings['rating'].unique())

10


[4.0, 5.0, 3.0, 2.0, 1.0, 4.5, 3.5, 2.5, 0.5, 1.5]

In [5]:
# Method from the Surprise library to load the DataFrame 
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Build the full trainset
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)
antitest = trainSet.build_anti_testset()

# Prueba 1: OK

In [94]:
from RBMAlgorithm import RBMAlgorithm
RBM = RBMAlgorithm(epochs=20)

In [95]:
# Fit the RBM model on the training set
RBM.fit(trainSet)

# Make predictions on the test set
predictions = RBM.test(testSet)

Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Processing user  0
Processing user  50
Processing user  100
Processing user  150
Processing user  200
Processing user  250
Processing user  300
Processing user  350
Processing user  400
Processing user  450
Processing user  500
Processing user  550
Processing user  600


In [96]:
predtest = RBM.test(testSet)
predantitest = RBM.test(antitest)

In [97]:
predantitest

[Prediction(uid=432, iid=474, r_ui=3.503229285466356, est=3.1030674, details={'was_impossible': False}),
 Prediction(uid=432, iid=4351, r_ui=3.503229285466356, est=2.8666945, details={'was_impossible': False}),
 Prediction(uid=432, iid=2987, r_ui=3.503229285466356, est=2.847082, details={'was_impossible': False}),
 Prediction(uid=432, iid=177, r_ui=3.503229285466356, est=2.6691315, details={'was_impossible': False}),
 Prediction(uid=432, iid=750, r_ui=3.503229285466356, est=3.1378028, details={'was_impossible': False}),
 Prediction(uid=432, iid=6503, r_ui=3.503229285466356, est=2.4360378, details={'was_impossible': False}),
 Prediction(uid=432, iid=8641, r_ui=3.503229285466356, est=3.1467571, details={'was_impossible': False}),
 Prediction(uid=432, iid=1203, r_ui=3.503229285466356, est=3.1601, details={'was_impossible': False}),
 Prediction(uid=432, iid=1179, r_ui=3.503229285466356, est=3.162779, details={'was_impossible': False}),
 Prediction(uid=432, iid=111362, r_ui=3.50322928546635

# Prueba 2: OK

Para preparar los datos de train y test, necesitamos crear conjuntos en formato de matriz con cada fila representando a un usuario y cada celda de la fila representando la valoración de cada película, dado que esta es la entrada esperada por el algoritmo RBM.

Para ello, necesitamos el número total de usuarios como números de fila y el número total de películas como número de columna
* num_users = trainSet.n_users
* num_movies = trainSet.n_items

In [6]:
def fitData(trainset):

    num_users = trainset.n_users
    num_movies = trainset.n_items

    # 3D matrix: users, movies and ratings
    # Ratings has size 10 given the possible rating values
    trainingMatrix = np.zeros([num_users, num_movies, 10], dtype=np.float32)

    for (uid, iid, rating) in trainset.all_ratings():
        adjustedRating = int(float(rating)*2.0) - 1
        trainingMatrix[int(uid), int(iid), adjustedRating] = 1

    # Flatten to a 2D array, with nodes for each possible rating type on each possible item, for every user.
    trainingMatrix = np.reshape(trainingMatrix, [trainingMatrix.shape[0], -1])

    return trainingMatrix

In [7]:
trainingMatrix = fitData(trainSet)
trainingMatrix.shape

(610, 89280)

In [8]:
"""
Updated on Sun Dec 1 08:32:13 2019

@author: Frank
@modified: Saurabh

@modified: Inés Gómez Fortis
"""

class RBM_v2(AlgoBase):

    def __init__(self, visibleDimensions, epochs=20, hiddenDimensions=50, ratingValues=10, learningRate=0.001, batchSize=100):
        AlgoBase.__init__(self)
        self.visibleDimensions = visibleDimensions
        self.epochs = epochs
        self.hiddenDimensions = hiddenDimensions
        self.ratingValues = ratingValues
        self.learningRate = learningRate
        self.batchSize = batchSize

    def fit(self, trainset, trainingMatrix):
        
        AlgoBase.fit(self, trainset)
        self.Train(trainingMatrix)
        
        num_users = trainset.n_users
        num_movies = trainset.n_items
               
        self.predictedRatings = np.zeros([num_users, num_movies], dtype=np.float32)
        for uiid in range(trainset.n_users):
            if (uiid % 50 == 0):
                print("Processing user ", uiid)
            recs = self.GetRecommendations([trainingMatrix[uiid]])
            recs = np.reshape(recs, [num_movies, 10])
            
            for itemID, rec in enumerate(recs):
                # The obvious thing would be to just take the rating with the highest score: rating = rec.argmax()
                # but this just leads to a huge multi-way tie for 5-star predictions.
                # Instead, the paper suggests performing normalization over K values to get probabilities and take the expectation as the prediction
                normalized = np.exp(rec)/np.sum(np.exp(rec), axis=0)
                rating = np.average(np.arange(10), weights=normalized)
                self.predictedRatings[uiid, itemID] = (rating + 1) * 0.5
        
        return self        
        

    # in order to reconstract any users rating for any item            
    def Train(self, X):

        # Initialize weights randomly (earlier versions of thie code had this block inside MakeGraph, but that was a bug.)
        maxWeight = -4.0 * np.sqrt(6.0 / (self.hiddenDimensions + self.visibleDimensions))
        self.weights = tf.Variable(tf.random.uniform([self.visibleDimensions, self.hiddenDimensions], minval=-maxWeight, maxval=maxWeight), tf.float32, name="weights")
        self.hiddenBias = tf.Variable(tf.zeros([self.hiddenDimensions], tf.float32, name="hiddenBias"))
        self.visibleBias = tf.Variable(tf.zeros([self.visibleDimensions], tf.float32, name="visibleBias"))

        for epoch in range(self.epochs):
            
            trX = np.array(X)
            for i in range(0, trX.shape[0], self.batchSize):
                epochX = trX[i:i+self.batchSize]
                self.MakeGraph(epochX)

            print("Trained epoch ", epoch)

    # get back rating predictions for a given user
    def GetRecommendations(self, inputUser):
        
        feed = self.MakeHidden(inputUser)
        rec = self.MakeVisible(feed)
        return rec[0]       

    def MakeGraph(self, inputUser):
        
        # Perform Gibbs Sampling for Contrastive Divergence, per the paper we assume k=1 instead of iterating over the 
        # forward pass multiple times since it seems to work just fine
        
        # Forward pass
        # Sample hidden layer given visible...
        # Get tensor of hidden probabilities
        hProb0 = tf.nn.sigmoid(tf.matmul(inputUser, self.weights) + self.hiddenBias)
        # Sample from all of the distributions
        hSample = tf.nn.relu(tf.sign(hProb0 - tf.random.uniform(tf.shape(hProb0))))
        # Stitch it together
        forward = tf.matmul(tf.transpose(inputUser), hSample)
        
        # Backward pass
        # Reconstruct visible layer given hidden layer sample
        v = tf.matmul(hSample, tf.transpose(self.weights)) + self.visibleBias
        
        # Build up our mask for missing ratings
        vMask = tf.sign(inputUser) # Make sure everything is 0 or 1
        # User + movie + indicator of movie been rated
        vMask3D = tf.reshape(vMask, [tf.shape(v)[0], -1, self.ratingValues]) # Reshape into arrays of individual ratings
        vMask3D = tf.reduce_max(vMask3D, axis=[2], keepdims=True) # Use reduce_max to either give us 1 for ratings that exist, and 0 for missing ratings
        
        # Extract rating vectors for each individual set of 10 rating binary values
        v = tf.reshape(v, [tf.shape(v)[0], -1, self.ratingValues])
        vProb = tf.nn.softmax(v * vMask3D) # Apply softmax activation function
        vProb = tf.reshape(vProb, [tf.shape(v)[0], -1]) # And shove them back into the flattened state. Reconstruction is done now.
        # vProb = reconstructed data

        # Stitch it together to define the backward pass and updated hidden biases
        hProb1 = tf.nn.sigmoid(tf.matmul(vProb, self.weights) + self.hiddenBias)
        backward = tf.matmul(tf.transpose(vProb), hProb1)
    
        # Now define what each epoch will do...
        # Run the forward and backward passes, and update the weights
        weightUpdate = self.weights.assign_add(self.learningRate * (forward - backward))
        # Update hidden bias, minimizing the divergence in the hidden nodes
        hiddenBiasUpdate = self.hiddenBias.assign_add(self.learningRate * tf.reduce_mean(hProb0 - hProb1, 0))
        # Update the visible bias, minimizng divergence in the visible results
        visibleBiasUpdate = self.visibleBias.assign_add(self.learningRate * tf.reduce_mean(inputUser - vProb, 0))

        self.update = [weightUpdate, hiddenBiasUpdate, visibleBiasUpdate]
        
    def MakeHidden(self, inputUser):
        hidden = tf.nn.sigmoid(tf.matmul(inputUser, self.weights) + self.hiddenBias)
        self.MakeGraph(inputUser)
        return hidden
    
    def MakeVisible(self, feed):
        visible = tf.nn.sigmoid(tf.matmul(feed, tf.transpose(self.weights)) + self.visibleBias)
        #self.MakeGraph(feed)
        return visible
    
    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')
        
        rating = self.predictedRatings[u, i]
        
        if (rating < 0.001):
            raise PredictionImpossible('No valid prediction exists.')
            
        return rating


In [9]:
# Create an RBM with (num items * rating values) visible nodes
model_v2 = RBM_v2(trainingMatrix.shape[1])

# Fit the RBM model on the training set
model_v2.fit(trainSet, trainingMatrix)

Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Processing user  0
Processing user  50
Processing user  100
Processing user  150
Processing user  200
Processing user  250
Processing user  300
Processing user  350
Processing user  400
Processing user  450
Processing user  500
Processing user  550
Processing user  600


In [10]:
predtest = model_v2.test(testSet)
predantitest = model_v2.test(antitest)

In [58]:
predtest

[Prediction(uid=140, iid=6765, r_ui=3.5, est=2.790531, details={'was_impossible': False}),
 Prediction(uid=603, iid=290, r_ui=4.0, est=3.0183632, details={'was_impossible': False}),
 Prediction(uid=438, iid=5055, r_ui=4.0, est=2.7207878, details={'was_impossible': False}),
 Prediction(uid=433, iid=164179, r_ui=5.0, est=3.1036127, details={'was_impossible': False}),
 Prediction(uid=474, iid=5114, r_ui=4.0, est=3.503229285466356, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=304, iid=1035, r_ui=4.0, est=3.0454543, details={'was_impossible': False}),
 Prediction(uid=298, iid=4974, r_ui=1.0, est=2.7883945, details={'was_impossible': False}),
 Prediction(uid=131, iid=293, r_ui=4.0, est=3.056513, details={'was_impossible': False}),
 Prediction(uid=288, iid=5784, r_ui=2.5, est=2.7545974, details={'was_impossible': False}),
 Prediction(uid=448, iid=97225, r_ui=2.5, est=2.817548, details={'was_impossible': False}),
 Prediction(uid=284, iid=585, r_ui

In [66]:
predantitest

[Prediction(uid=432, iid=474, r_ui=3.503229285466356, est=3.0600433, details={'was_impossible': False}),
 Prediction(uid=432, iid=4351, r_ui=3.503229285466356, est=2.8392367, details={'was_impossible': False}),
 Prediction(uid=432, iid=2987, r_ui=3.503229285466356, est=2.9117796, details={'was_impossible': False}),
 Prediction(uid=432, iid=177, r_ui=3.503229285466356, est=2.7116396, details={'was_impossible': False}),
 Prediction(uid=432, iid=750, r_ui=3.503229285466356, est=3.1083212, details={'was_impossible': False}),
 Prediction(uid=432, iid=6503, r_ui=3.503229285466356, est=2.5628557, details={'was_impossible': False}),
 Prediction(uid=432, iid=8641, r_ui=3.503229285466356, est=3.0988302, details={'was_impossible': False}),
 Prediction(uid=432, iid=1203, r_ui=3.503229285466356, est=3.131605, details={'was_impossible': False}),
 Prediction(uid=432, iid=1179, r_ui=3.503229285466356, est=3.058539, details={'was_impossible': False}),
 Prediction(uid=432, iid=111362, r_ui=3.50322928546

En teoría solo usa los datos conocidos para training y va reconstruyendo el resto (sparse data), no?

In [83]:
#ratings[ratings['userId'] == 432]
ratings[(ratings['userId'] == 432) & (ratings['movieId'] == 474)]

,userId,movieId,rating,timestamp


# Metrics

In [11]:
import os
os.chdir('../metrics')
from metrics import evaluationMetrics
em = evaluationMetrics()

# Check

In [84]:
from collections import defaultdict

def getTopN(predictions, n=10, minimumRating=4.0):
    """Returns the top N recommended movies for each user based on estimated ratings.

    Args:
        predictions (list of tuples): A list of predictions where each tuple contains a user ID,
            a movie ID, an actual rating, an estimated rating, and some additional information.
        n (int, optional): The number of top recommended movies to return for each user.
            Defaults to 10.
        minimumRating (float, optional): The minimum estimated rating for a movie to be considered
            a top recommendation. Defaults to 4.0.

    Returns:
        defaultdict: A defaultdict where the keys are user IDs and the values are lists of tuples,
            where each tuple contains a movie ID and its estimated rating. The length of each list
            is equal to the specified value of `n` or less if there are not enough movies that meet
            the minimum rating threshold.
    """
    topN = defaultdict(list)
    
    max_est = 0 #check

    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        #print("dentro")
        #print(estimatedRating)
        if estimatedRating > max_est:
            max_est = estimatedRating
            
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    print(max_est)
    return topN

In [85]:
# Get top N
top = getTopN(predantitest, minimumRating = 3.5)

3.2069242


In [88]:
top

defaultdict(list, {})

In [89]:
# Get top N
top_v2 = getTopN(predantitest, minimumRating = 3.0)

3.2069242


In [90]:
top_v2

defaultdict(list,
            {432: [(913, 3.1957064),
              (112552, 3.1852684),
              (914, 3.17505),
              (1249, 3.1749468),
              (1276, 3.1728954),
              (1244, 3.172652),
              (31658, 3.171748),
              (81845, 3.1705246),
              (1233, 3.1704633),
              (44555, 3.169145)],
             288: [(913, 3.1961968),
              (112552, 3.1858046),
              (60684, 3.1763148),
              (914, 3.1754985),
              (1249, 3.175394),
              (1244, 3.173034),
              (31658, 3.1722271),
              (81845, 3.171032),
              (1233, 3.1710203),
              (44555, 3.169677)],
             599: [(60684, 3.17693),
              (914, 3.1759453),
              (1249, 3.1758397),
              (1276, 3.1735084),
              (1244, 3.1734142),
              (44555, 3.1702073),
              (57669, 3.1665385),
              (84152, 3.1649203),
              (69481, 3.1646466),
        

In [93]:
novelty_v2 = em.getNovelty(top_v2,trainSet)
novelty_v2

655.3877049180328

**Fin del CHECK**

In [50]:
# Get test and antitest predictions
# predtest = model.test(testSet)
# predantitest = model.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_RBM = em.getTopN(predantitest, minimumRating = 3.5)

In [57]:
top_10_RBM

defaultdict(list,
            {0: [],
             1: [],
             2: [],
             3: [],
             4: [],
             5: [],
             6: [],
             7: [],
             8: [],
             9: [],
             10: [],
             11: [],
             12: [],
             13: [],
             14: [],
             15: [],
             16: [],
             17: [],
             18: [],
             19: [],
             20: [],
             21: [],
             22: [],
             23: [],
             24: [],
             25: [],
             26: [],
             27: [],
             28: [],
             29: [],
             30: [],
             31: [],
             32: [],
             33: [],
             34: [],
             35: [],
             36: [],
             37: [],
             38: [],
             39: [],
             40: [],
             41: [],
             42: [],
             43: [],
             44: [],
             45: [],
             46: [],
     

In [12]:
# Get top N recommended movies for each user based on estimated ratings
top_10_RBM = em.getTopN(predantitest, minimumRating = 3.0)

## Métricas de precisión: RMSE y MAE

In [13]:
# # RMSE
rmse = accuracy.rmse(predtest)

# MAE
mae = accuracy.mae(predtest)

RMSE: 1.1620
MAE:  0.9641


## Métricas de relevancia: Precision, Recall y NDCG

In [14]:
# Precision
precisions = em.getPrecision(predtest, k=10, threshold=3.5)

# Mean Average Precision
mapModel = np.mean(list(precisions.values()))

# Recall
recalls = em.getRecall(predtest, k=10, threshold=3.5)

# Mean Average Recall
marModel = np.mean(list(recalls.values()))

# Normalized discounted cummulative gain (NDCG)
ndcgs, mean_ndcg = em.getNDCG(predtest,10)

## Otras métricas de interés: Coverage, User Coverage y Novelty

In [15]:
# Coverage
coverage = em.getCoverage(top_10_RBM,trainSet.n_items,trainSet.all_users())

# User coverage
user_coverage = em.getUserCoverage(top_10_RBM, trainSet.n_users,4)

# Novelty
novelty = em.getNovelty(top_10_RBM,trainSet)

Por último creamos un dataframe con todas las métricas de evaluación asociadas al modelo

In [16]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "RBM", "RMSE": rmse, "MAE": mae, "MAP": mapModel, "MAR": marModel,
                     "Mean_NDCG": mean_ndcg, "Coverage": coverage, "User_Coverage": user_coverage,
                     "Novelty": novelty})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

,Model,RMSE,MAE,MAP,MAR,Mean_NDCG,Coverage,User_Coverage,Novelty
0,RBM_v2,1.16202,0.96412,0.163041,0.007933,0.935966,0.0028,0.0,667.055574


In [17]:
# Add the results to the dataframe with the metrics of all models.
em.addToMetricsDataframe(metrics_df)

# Random Algorithm

Vamos a evaluar también un modelo Random, en concreto "NormalPredictor" para poder comparar sus resultados con el resto de modelos. NormalPredictor es un algoritmo simple en Surprise que predice calificaciones aleatoriamente basado en la distribución del conjunto de entrenamiento. Supone una distribución normal de las calificaciones y genera predicciones aleatorias en función de esa distribución.

In [ ]:
# Create the model
Random = NormalPredictor()
Random.fit(trainSet)

In [ ]:
# Get test and antitest predictions
predtest_random = Random.test(testSet)
predantitest_random = Random.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_random = em.getTopN(predantitest_random,minimumRating = 3.5)

## Compute metrics

In [ ]:
# Accuracy Metrics
rmse_random = accuracy.rmse(predtest_random)
mae_random = accuracy.mae(predtest_random)

# Relevance metrics
precisions_random = em.getPrecision(predtest_random, k=10, threshold=3.5)
mapModel_random = np.mean(list(precisions_random.values()))

recalls_random = em.getRecall(predtest_random, k=10, threshold=3.5)
marModel_random = np.mean(list(recalls_random.values()))

ndcgs_random, mean_ndcg_random = em.getNDCG(predtest_random,10)

# Other metrics
coverage_random = em.getCoverage(top_10_random,trainSet.n_items,trainSet.all_users())
user_coverage_random = em.getUserCoverage(top_10_random, trainSet.n_users,4)
novelty_random = em.getNovelty(top_10_random,trainSet)

In [ ]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "random", "RMSE": rmse_random, "MAE": mae_random, "MAP": mapModel_random, "MAR": marModel_random,
                     "Mean_NDCG": mean_ndcg_random, "Coverage": coverage_random, "User_Coverage": user_coverage_random,
                     "Novelty": novelty_random})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

In [ ]:
# Add the results to the dataframe with the metrics of all models.
em.addToMetricsDataframe(metrics_df)